In [1]:
import pandas as pd

df = pd.read_csv('kkbox_data/kkbox_data.csv')
df.head()

,rank,artist,artist_id,song_id,album_id,date,category
0,1,五月天 (Mayday),9XN-7yg5vg3gYnCdsM,5_DAFKxzMf04hZqXk4,0le7fKde-BNMwoAst3,2021-01-01,mandarin
1,2,盧廣仲 (Crowd Lu),8qCONzjl89Yak9KFxZ,L_Aag3up_NyBpahTJG,4rzv06iAFoOyareMC7,2021-01-01,mandarin
2,3,任然,#,#,#,2021-01-01,mandarin
3,4,艾薇,SkYfuVHgT10-nfv3Tz,XXONe3fIN4kbrmzJqh,XXDPJ346EjhjDZXb-W,2021-01-01,mandarin
4,5,aMEI (張惠妹),Ok6kf9NM0d0grGYNbl,P_c_y1B6adNriKJkm6,Okl8SAlaH9M2s97GAv,2021-01-01,mandarin


In [2]:
album_id = df[df['album_id'] != '#']['album_id'].drop_duplicates()
album_id

0       0le7fKde-BNMwoAst3
1       4rzv06iAFoOyareMC7
3       XXDPJ346EjhjDZXb-W
4       Okl8SAlaH9M2s97GAv
5       D_QwkPGasaFcNVcZCz
               ...        
8788    HYj_0bnwU1EtWSkqVG
8798    9a2zra3DUhQp-Ssz_o
9075    Creg7E-5yWtogi7IJB
9127    GlrhpaVAs-18Awx0VL
9136    -rIWDmb51XBjJ7CTLp
Name: album_id, Length: 368, dtype: object

In [4]:
# get token
from kkbox_developer_sdk.auth_flow import KKBOXOAuth
import http.client


# Replace CLIENT_ID and CLIENT_SECRET with your ID and Secret
auth = KKBOXOAuth('5e5e2e22f6e9940a106b3befde703767', 'd3aa398ae0aa2018b5205a6a98692a4d')
token = auth.fetch_access_token_by_client_credentials()
access_token = token.access_token

conn = http.client.HTTPSConnection("api.kkbox.com")
headers = {
    'accept': "application/json",
    'authorization': f"Bearer {access_token}"
}

In [5]:
import json
from tqdm import tqdm
import time


# 創建一個列表來存儲所有歌曲的數據
all_tracks = []

for id in tqdm(album_id):
    try:
        conn.request("GET", f"/v1.1/albums/{id}/tracks?territory=TW&offset=0&limit=500", headers=headers)
        res = conn.getresponse()
        data = res.read()
        
        # Parse the JSON response
        track_data = json.loads(data.decode("utf-8"))
        
        # 提取每首歌曲的信息
        for track in track_data['data']:
            all_tracks.append({
                'name': track['name'],
                'id': track['id'],
                'duration': track['duration'],
                'track_number': track['track_number'],
                'album_id': id  # 添加專輯ID以便追蹤
            })
        
        # 添加一個小延遲以避免過度請求API
        time.sleep(0.1)
    
    except Exception as e:
        print(f"處理專輯ID {id} 時發生錯誤: {str(e)}")

# 將所有數據轉換為DataFrame
df_track = pd.DataFrame(all_tracks)

100%|██████████| 368/368 [01:40<00:00,  3.66it/s]


In [6]:
df_track

,name,id,duration,track_number,album_id
0,因為你 所以我,5_DAFKxzMf04hZqXk4,281808.0,1,0le7fKde-BNMwoAst3
1,刻在我心底的名字 - 電影〈刻在你心底的名字〉主題曲,L_Aag3up_NyBpahTJG,320182.0,1,4rzv06iAFoOyareMC7
2,失重前幸福,XXONe3fIN4kbrmzJqh,222484.0,1,XXDPJ346EjhjDZXb-W
3,緩緩,P_c_y1B6adNriKJkm6,227343.0,1,Okl8SAlaH9M2s97GAv
4,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,4kDxlI5PWkaL8-hS8a,259604.0,1,D_QwkPGasaFcNVcZCz
...,...,...,...,...,...
2456,逐夢孟加拉,TY-_rcqlGziN6FEixf,281286.0,8,-rIWDmb51XBjJ7CTLp
2457,今夜就這樣 - 日劇<非獸性男女>主題曲,Km7sGyEIR1boO0wt7o,238106.0,9,-rIWDmb51XBjJ7CTLp
2458,哪怕明天世界就會結束 - 電影<相對世界。明日終結？>主題曲,CmjOujwltBWGrqfr_g,262739.0,10,-rIWDmb51XBjJ7CTLp
2459,GOOD NIGHT BABY,H_dfH79o6Jmyze3bpm,330657.0,11,-rIWDmb51XBjJ7CTLp
